In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# from transformers.utils import logging as hf_logging
# hf_logging.set_verbosity_error() 

import sys
import random
from pathlib import Path
project_root = Path().resolve().parent
sys.path.append(str(project_root / "src"))
from tqdm.notebook import tqdm

from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import make_scorer, f1_score, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from itertools import product

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from torch.utils.data import DataLoader
from dataset.custom_data import IMDBDataset
from models.lstm import GloVeLSTM
from utils.embeddings import load_glove_embeddings


from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_seed(42)

In [4]:
df = pd.read_parquet(project_root / "data" / "imdb_reviews.parquet")
df = df[["review", "sentiment"]].dropna().reset_index(drop=True)

X = df["review"].astype(str).to_list()
y = df["sentiment"].astype(int).to_numpy()

In [5]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score)

pipe = Pipeline([
    ("vec", TfidfVectorizer(min_df=5, ngram_range=(1,2), sublinear_tf=True)),
    ("clf", LinearSVC())
])


In [6]:
param_grid = [
    {  # LinearSVC branch
        "clf": [LinearSVC()],
        "vec__min_df": [3, 5, 10],
        "vec__max_features": [40000, 60000, None],
        "vec__ngram_range": [(1,1), (1,2)],
        "clf__C": [0.5, 1.0, 2.0],
    },
    {  # SGDClassifier branch
        "clf": [SGDClassifier(random_state=42)],
        "vec__min_df": [3, 5],
        "vec__ngram_range": [(1,1), (1,2)],
        "clf__loss": ["hinge", "log_loss"],
        "clf__alpha": [1e-5, 1e-4, 1e-3],
        "clf__penalty": ["l2", "l1"],
        "clf__max_iter": [1000, 2000],
    }
]

gs = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True
)
gs.fit(X, y)

print("Best F1:", gs.best_score_)
print("Best params:", gs.best_params_)
best_model = gs.best_estimator_

Fitting 5 folds for each of 150 candidates, totalling 750 fits
Best F1: 0.9110504394179648
Best params: {'clf': LinearSVC(), 'clf__C': 0.5, 'vec__max_features': None, 'vec__min_df': 3, 'vec__ngram_range': (1, 2)}


In [7]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total, loss_sum, correct = 0, 0.0, 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.float().to(device)
        optimizer.zero_grad()
        logits = model(xb)               # (B,)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        loss_sum += float(loss.item()) * yb.size(0)
        preds = (logits.sigmoid() >= 0.5).long()
        correct += (preds == yb.long()).sum().item()
        total += yb.size(0)
    return loss_sum/total, correct/total

@torch.no_grad()
def eval_epoch(model, loader, criterion):
    model.eval()
    total, loss_sum, correct = 0, 0.0, 0
    ys, ps = [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.float().to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        loss_sum += float(loss.item()) * yb.size(0)

        prob = logits.sigmoid()
        pred = (prob >= 0.5).long()
        correct += (pred == yb.long()).sum().item()
        total += yb.size(0)

        ys.append(yb.cpu().numpy())
        ps.append(pred.cpu().numpy())

    y = np.concatenate(ys)
    p = np.concatenate(ps)
    return loss_sum/total, correct/total, y, p


In [8]:
def train_lstm_one_fold(params, tr_idx, va_idx, texts, labels, glove_filename: str):
    tr_texts = [texts[i] for i in tr_idx]
    tr_labels = [labels[i] for i in tr_idx]
    va_texts = [texts[i] for i in va_idx]
    va_labels = [labels[i] for i in va_idx]

    # Build train fold dataset (creates vocab from train fold)
    train_ds = IMDBDataset(
        texts=tr_texts,
        labels=tr_labels,
        max_len=params["max_len"],
        preprocess=True,
        min_freq=params.get("min_freq", 2),
        max_vocab_size=params.get("max_vocab_size", 30000),
        language="english"
    )

    # Validation dataset shares the same vocab
    val_ds = IMDBDataset(
        texts=va_texts,
        labels=va_labels,
        max_len=params["max_len"],
        preprocess=True,
        min_freq=params.get("min_freq", 2),
        max_vocab_size=params.get("max_vocab_size", 30000),
        language="english"
    )
    val_ds.vocab = train_ds.vocab  # align token->id mapping

    train_loader = DataLoader(train_ds, batch_size=params["batch_size"], shuffle=True, num_workers=0)
    val_loader   = DataLoader(val_ds,   batch_size=params["batch_size"], shuffle=False, num_workers=0)

    # Load GloVe embeddings for THIS fold's vocab (OOV get random)
    glove_tensor = load_glove_embeddings(glove_filename, train_ds.vocab, embedding_dim=params["emb_dim"])

    # Build model
    model = GloVeLSTM(
                vocab_size=len(train_ds.vocab),
                emb_dim=params["emb_dim"],
                hidden_dim=params["hidden_dim"],
                num_layers=params["num_layers"],
                bidirectional=params["bidirectional"],
                dropout=params["dropout"],
                pad_idx=0,
                pretrained_embeddings=glove_tensor
            ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
    criterion = nn.BCEWithLogitsLoss()

    # Early stopping
    patience = params.get("patience", 2)
    best_acc, best_f1, best_epoch = 0.0, 0.0, None
    patience_ctr = 0

    for epoch in range(1, params["epochs"] + 1):
        tr_loss, tr_acc = train_epoch(model, train_loader, optimizer, criterion)
        va_loss, va_acc, y_true, y_pred = eval_epoch(model, val_loader, criterion)
        va_f1 = f1_score(y_true, y_pred)

        tqdm.write(f"[Fold] Ep{epoch:02d} | tr_loss={tr_loss:.4f} tr_acc={tr_acc:.4f} "
                   f"| va_loss={va_loss:.4f} va_acc={va_acc:.4f} va_f1={va_f1:.4f}")

        if va_acc > best_acc:
            best_acc = va_acc
            best_f1  = va_f1
            best_epoch = epoch
            patience_ctr = 0
        else:
            patience_ctr += 1
            if patience_ctr >= patience:
                tqdm.write(f"Early stop @ epoch {epoch} (best acc={best_acc:.4f} f1={best_f1:.4f})")
                break

    return {"val_acc": best_acc, "val_f1": best_f1, "best_epoch": best_epoch}

In [9]:
def kfold_lstm_cv(texts, labels, param_grid, glove_filename="glove.6B.100d.txt", n_splits=5, seed=42):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    X = np.array(texts, dtype=object)
    y = np.array(labels, dtype=int)

    all_runs = []
    for params in param_grid:
        fold_metrics = []
        tqdm.write(f"\n=== Params: {params} ===")
        for fold, (tr_idx, va_idx) in enumerate(cv.split(X, y), start=1):
            tqdm.write(f"\n--- Fold {fold}/{n_splits} ---")
            set_seed(42 + fold)  # fold-stable seed
            m = train_lstm_one_fold(params, tr_idx, va_idx, texts, labels, glove_filename)
            fold_metrics.append(m)

        mean_acc = float(np.mean([m["val_acc"] for m in fold_metrics]))
        std_acc  = float(np.std([m["val_acc"] for m in fold_metrics]))
        mean_f1  = float(np.mean([m["val_f1"] for m in fold_metrics]))
        std_f1   = float(np.std([m["val_f1"] for m in fold_metrics]))

        all_runs.append({
            "params": params,
            "mean_acc": mean_acc, "std_acc": std_acc,
            "mean_f1": mean_f1,   "std_f1": std_f1,
        })

        tqdm.write(f"\nRESULT | acc={mean_acc:.4f}±{std_acc:.4f} | f1={mean_f1:.4f}±{std_f1:.4f}")

    all_runs = sorted(all_runs, key=lambda d: (d["mean_f1"], d["mean_acc"]), reverse=True)
    return all_runs

In [10]:
texts = df["review"].astype(str).tolist()
labels = df["sentiment"].astype(int).tolist()

# Small param grid to start (expand once it runs fine)
param_grid = [
    {
        "emb_dim": 100,          # must match your GloVe file (e.g., glove.6B.100d.txt)
        "hidden_dim": 128,
        "num_layers": 2,
        "bidirectional": True,
        "dropout": 0.4,
        "lr": 1e-3,
        "batch_size": 64,
        "epochs": 8,
        "max_len": 256,
        "min_freq": 2,
        "max_vocab_size": 30000,
        "patience": 2,
    },
    {
        "emb_dim": 100,
        "hidden_dim": 256,
        "num_layers": 2,
        "bidirectional": True,
        "dropout": 0.5,
        "lr": 5e-4,
        "batch_size": 64,
        "epochs": 8,
        "max_len": 256,
        "min_freq": 2,
        "max_vocab_size": 30000,
        "patience": 2,
    },
]

# Run CV
results = kfold_lstm_cv(
    texts=X,
    labels=y,
    param_grid=param_grid,
    glove_filename="glove.6B.100d.txt",  # file must be in <project_root>/data
    n_splits=5,
    seed=42
)

# Leaderboard
pd.DataFrame(results)


=== Params: {'emb_dim': 100, 'hidden_dim': 128, 'num_layers': 2, 'bidirectional': True, 'dropout': 0.4, 'lr': 0.001, 'batch_size': 64, 'epochs': 8, 'max_len': 256, 'min_freq': 2, 'max_vocab_size': 30000, 'patience': 2} ===

--- Fold 1/5 ---
[Fold] Ep01 | tr_loss=0.5953 tr_acc=0.6805 | va_loss=0.4347 va_acc=0.8027 va_f1=0.7934
[Fold] Ep02 | tr_loss=0.3932 tr_acc=0.8293 | va_loss=0.3630 va_acc=0.8424 va_f1=0.8362
[Fold] Ep03 | tr_loss=0.3396 tr_acc=0.8545 | va_loss=0.3365 va_acc=0.8572 va_f1=0.8631
[Fold] Ep04 | tr_loss=0.3105 tr_acc=0.8683 | va_loss=0.3240 va_acc=0.8604 va_f1=0.8540
[Fold] Ep05 | tr_loss=0.2861 tr_acc=0.8806 | va_loss=0.2977 va_acc=0.8752 va_f1=0.8769
[Fold] Ep06 | tr_loss=0.2682 tr_acc=0.8902 | va_loss=0.3197 va_acc=0.8705 va_f1=0.8643
[Fold] Ep07 | tr_loss=0.2474 tr_acc=0.8989 | va_loss=0.2774 va_acc=0.8843 va_f1=0.8866
[Fold] Ep08 | tr_loss=0.2270 tr_acc=0.9092 | va_loss=0.2980 va_acc=0.8819 va_f1=0.8868

--- Fold 2/5 ---
[Fold] Ep01 | tr_loss=0.5924 tr_acc=0.6938 |

,params,mean_acc,std_acc,mean_f1,std_f1
0,"{'emb_dim': 100, 'hidden_dim': 128, 'num_layer...",0.883829,0.006560,0.884136,0.006506
1,"{'emb_dim': 100, 'hidden_dim': 256, 'num_layer...",0.879271,0.004578,0.879814,0.006981


In [11]:
model_name = "distilbert-base-uncased"  # faster than BERT-base
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=256)

def train_eval_one_fold(train_idx, val_idx, hp):
    tr_texts = [texts[i] for i in train_idx]; tr_labels = [labels[i] for i in train_idx]
    va_texts = [texts[i] for i in val_idx];   va_labels = [labels[i] for i in val_idx]

    train_ds = Dataset.from_dict({"text": tr_texts, "label": tr_labels}).map(tokenize_batch, batched=True)
    val_ds   = Dataset.from_dict({"text": va_texts, "label": va_labels}).map(tokenize_batch, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    args = TrainingArguments(
        output_dir="tmp_out",
        learning_rate=hp["lr"],
        per_device_train_batch_size=hp["bsz"],
        per_device_eval_batch_size=hp["bsz"],
        num_train_epochs=hp["epochs"],
        weight_decay=hp["wd"],
        eval_strategy="epoch",
        disable_tqdm=False,
        save_strategy="no",
        log_level="error",
        log_level_replica="error",
        logging_steps=50,
        report_to=[],
        fp16=torch.cuda.is_available(),
    )

    def compute_metrics(eval_pred):
        logits, y_true = eval_pred
        y_pred = np.argmax(logits, axis=1)
        return {"accuracy": (y_pred == y_true).mean(), "f1": f1_score(y_true, y_pred)}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    out = trainer.evaluate()
    preds = trainer.predict(val_ds)
    y_pred = np.argmax(preds.predictions, axis=1)
    y_true = np.array(va_labels)
    return {"eval_accuracy": out["eval_accuracy"], "eval_f1": out["eval_f1"], "y_true": y_true, "y_pred": y_pred}


In [13]:
def expand_grid(grid_dict):
    keys = list(grid_dict.keys())
    vals = list(grid_dict.values())
    return [dict(zip(keys, combo)) for combo in product(*vals)]

grid = {
    "lr":   [2e-5, 3e-5, 5e-5],
    "bsz":  [16, 32, 64],
    "epochs": [2, 3, 4],        
    "wd":   [0.01],
}
hp_list = expand_grid(grid)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
all_rows = []           # one row per (hp, fold)
agg_preds = {}          # (hp_idx) -> list of (y_true, y_pred) for confusion matrices

for hp_idx, hp in enumerate(hp_list):
    agg_preds[hp_idx] = []
    for fold_id, (tr_idx, va_idx) in enumerate(cv.split(X, y), start=1):
        m = train_eval_one_fold(tr_idx, va_idx, hp)
        all_rows.append({
            "hp_idx": hp_idx,
            "fold": fold_id,
            "lr": hp["lr"],
            "bsz": hp["bsz"],
            "epochs": hp["epochs"],
            "wd": hp["wd"],
            "acc": m["eval_accuracy"],
            "f1":  m["eval_f1"],
        })
        agg_preds[hp_idx].append( (m["y_true"], m["y_pred"]) )

results_df = pd.DataFrame(all_rows)

Map:   0%|          | 0/39665 [00:00<?, ? examples/s]

Map:   0%|          | 0/9917 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.254500,0.282800,0.891600,0.885723
2,0.161400,0.274318,0.910154,0.910533


Map:   0%|          | 0/39665 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9917 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.264500,0.261105,0.892407,0.888307
2,0.192400,0.245127,0.908843,0.908981
3,0.126200,0.257777,0.907533,0.908125


Map:   0%|          | 0/39665 [00:00<?, ? examples/s]

Map:   0%|          | 0/9917 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.255800,0.227808,0.909045,0.907790
2,0.192500,0.215698,0.913079,0.914382


In [ ]:
summary = (
    results_df
    .groupby(["hp_idx","lr","bsz","epochs","wd"], as_index=False)
    .agg(mean_acc=("acc","mean"), std_acc=("acc","std"),
         mean_f1=("f1","mean"),   std_f1=("f1","std"))
    .sort_values(["mean_f1","mean_acc"], ascending=False)
    .reset_index(drop=True)
)

display(summary)

# ---- pick best & worst by mean_f1
best_row  = summary.iloc[0]
worst_row = summary.iloc[-1]
best_hp_idx  = int(best_row["hp_idx"])
worst_hp_idx = int(worst_row["hp_idx"])

print("Best HP:",  dict(best_row[["lr","bsz","epochs","wd"]]))
print("Worst HP:", dict(worst_row[["lr","bsz","epochs","wd"]]))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(results_df.loc[results_df.hp_idx==best_hp_idx,"fold"],
        results_df.loc[results_df.hp_idx==best_hp_idx,"acc"], marker="o", label="best (acc)")
ax.plot(results_df.loc[results_df.hp_idx==worst_hp_idx,"fold"],
        results_df.loc[results_df.hp_idx==worst_hp_idx,"acc"], marker="o", label="worst (acc)")
ax.set_title("Per-fold accuracy: best vs worst hyperparams")
ax.set_xlabel("fold"); ax.set_ylabel("accuracy"); ax.set_xticks([1,2,3,4,5]); ax.legend()
plt.tight_layout(); plt.show()

# ---- aggregate confusion matrices (sum over folds) for best vs worst
def sum_conf_mat(pairs):
    cm_sum = np.zeros((2,2), dtype=int)
    for y_true, y_pred in pairs:
        cm_sum += confusion_matrix(y_true, y_pred, labels=[0,1])
    return cm_sum

cm_best  = sum_conf_mat(agg_preds[best_hp_idx])
cm_worst = sum_conf_mat(agg_preds[worst_hp_idx])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,4))
for ax, cm, title in [(axes[0], cm_best,  "Best HP (sum over folds)"),
                      (axes[1], cm_worst, "Worst HP (sum over folds)")]:
    disp = ConfusionMatrixDisplay(cm, display_labels=["neg","pos"])
    disp.plot(ax=ax, cmap="Blues", values_format="d", colorbar=False)
    ax.set_title(title)
plt.tight_layout(); plt.show()